In [17]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp, ttest_ind, f_oneway, chi2_contingency, pearsonr, shapiro
from scipy.stats import norm

# Definir o nível de significância
alpha = 0.05

# Carregar os dados dos experimentos A/B
caminho_arquivo_ab = '/content/emj7sgVaSciH6XGn46Cd_experimento_teste_ab (1).csv'
experimento_ab = pd.read_csv(caminho_arquivo_ab)

# Carregar os dados dos pacientes
caminho_arquivo_pacientes = '/content/6SdyVe6iQWueKQRwdqCI_pacientes.csv'
dados_pacientes = pd.read_csv(caminho_arquivo_pacientes)

# Questão 1: Qual dos cenários tem a maior taxa de conversão?

# Calcular a taxa de conversão para cada versão da página A/B
taxa_conversao_A = experimento_ab[experimento_ab['Versão_Página'] == 'A']['Conversões'].mean()
taxa_conversao_B = experimento_ab[experimento_ab['Versão_Página'] == 'B']['Conversões'].mean()

# Comparar as taxas de conversão das duas versões da página
if taxa_conversao_A > taxa_conversao_B:
    print("A versão A tem a maior taxa de conversão.")
elif taxa_conversao_B > taxa_conversao_A:
    print("A versão B tem a maior taxa de conversão.")
else:
    print("As duas versões têm a mesma taxa de conversão.")

# Questão 2: Calcule qual o tamanho da amostra necessária para o teste A/B

# Definir os parâmetros
alpha = 0.05
power = 0.80
p0 = taxa_conversao_A  # Taxa de conversão atual do cenário A
p1 = taxa_conversao_A + 0.10  # Taxa de conversão desejada (aumento de 10%)

# Calculando o tamanho da amostra
effect_size = np.abs(p1 - p0) / np.sqrt((p0 * (1 - p0)) + (p1 * (1 - p1)))
nobs = NormalIndPower().solve_power(effect_size=effect_size, alpha=alpha, power=power, alternative='larger')

# Arredondando para o próximo número inteiro
nobs = int(np.ceil(nobs))

print("Tamanho da amostra necessário:", nobs)

# Questão 3: Teste de hipótese sobre a idade média dos pacientes com problemas cardíacos

# Definir a hipótese nula
media_esperada = 50

# Selecionar uma amostra aleatória de pacientes com problemas cardíacos
amostra_idades = dados_pacientes.loc[[909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967, 699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362], 'Idade']

# Teste t de uma amostra para média
statistic, p_valor = ttest_1samp(amostra_idades, media_esperada, alternative='greater')

# Verificar se rejeitamos ou não a hipótese nula
if p_valor < alpha:
    print("Rejeitamos a hipótese nula. Há evidências estatísticas de que a idade média das pessoas com problemas cardíacos é maior que 50 anos.")
else:
    print("Não rejeitamos a hipótese nula. Não há evidências estatísticas suficientes para concluir que a idade média das pessoas com problemas cardíacos é maior que 50 anos.")

# Questão 4: Determinar se a amostra é dependente ou independente

# Como os conjuntos são divididos em duas categorias independentes, a amostra é independente.

# Questão 5: Comparar a pressão arterial média entre pacientes com e sem condições de saúde adicionais

# Amostra de pacientes com condições de saúde adicionais
amostra_com_condicoes = dados_pacientes.loc[[690, 894, 67, 201, 364, 19, 60, 319, 588, 643, 855, 623, 530, 174, 105, 693, 6, 462, 973, 607, 811, 346, 354, 966, 943, 372], 'Pressao_Arterial']

# Amostra de pacientes saudáveis
amostra_saudaveis = dados_pacientes.loc[[909, 751, 402, 400, 726, 39, 184, 269, 255, 769, 209, 715, 677, 381, 793, 697, 89, 280, 232, 756, 358, 36, 439, 768, 967, 699, 473, 222, 89, 639, 883, 558, 757, 84, 907, 895, 217, 224, 311, 348, 146, 505, 273, 957, 362], 'Pressao_Arterial']

# Teste t de duas amostras independentes
statistic, p_valor = ttest_ind(amostra_com_condicoes, amostra_saudaveis)

# Verificar se rejeitamos ou não a hipótese nula
if p_valor < alpha:
    print("Rejeitamos a hipótese nula. Há evidências de que a pressão arterial média para pacientes com condições de saúde adicionais é diferente da pressão arterial média para pacientes saudáveis.")
else:
    print("Não rejeitamos a hipótese nula. Não há evidências de que a pressão arterial média para pacientes com condições de saúde adicionais é diferente da pressão arterial média para pacientes saudáveis.")

# Questão 6: Teste ANOVA para verificar se há diferença significativa na pressão arterial média entre grupos étnicos

# Agrupar dados por etnia e extrair as amostras
amostras_etnicas = [dados_pacientes[dados_pacientes['Etnia'] == etnia]['Pressao_Arterial'] for etnia in dados_pacientes['Etnia'].unique()]

# Teste ANOVA
statistic, p_valor = f_oneway(*amostras_etnicas)

# Verificar se rejeitamos ou não a hipótese nula
if p_valor < alpha:
    print("Rejeitamos a hipótese nula. Há evidências de que há uma diferença significativa na pressão arterial média entre diferentes grupos étnicos nesta população.")
else:
    print("Não rejeitamos a hipótese nula. Não há evidências de que há uma diferença significativa na pressão arterial média entre diferentes grupos étnicos nesta população.")

# Questão 7: Teste qui-quadrado para entender a relação entre o sexo e condições de saúde adicionais

# Tabela de contingência
contingency_table = pd.crosstab(dados_pacientes['Genero'], dados_pacientes['Estado_Saude'])

# Teste qui-quadrado
chi2, p_valor, _, _ = chi2_contingency(contingency_table)

# Verificar se rejeitamos ou não a hipótese nula
if p_valor < alpha:
    print("Rejeitamos a hipótese nula. Há uma relação significativa entre o sexo e condições de saúde adicionais.")
else:
    print("Não rejeitamos a hipótese nula. Não há evidências de que há uma relação significativa entre o sexo e condições de saúde adicionais.")

# Questão 8: Teste de correlação entre idade dos pacientes e pressão arterial

# Teste de correlação de Pearson
correlacao, p_valor = pearsonr(dados_pacientes['Idade'], dados_pacientes['Pressao_Arterial'])

# Verificar se rejeitamos ou não a hipótese nula
if p_valor < alpha:
    print("Rejeitamos a hipótese nula. Há evidências de que há uma associação entre a idade dos pacientes e sua pressão arterial.")
else:
    print("Não rejeitamos a hipótese nula. Não há evidências de que há uma associação entre a idade dos pacientes e sua pressão arterial.")

# Questão 9: Intervalo de confiança para a média da pressão arterial entre pacientes com condições de saúde adicionais

# Calcular o intervalo de confiança
conf_int = norm.interval(0.95, loc=amostra_com_condicoes.mean(), scale=amostra_com_condicoes.std())

print("Intervalo de confiança para a média da pressão arterial entre pacientes com condições de saúde adicionais:", conf_int)

# Questão 10: Teste de normalidade da distribuição da pressão arterial na população

# Teste de Shapiro-Wilk para normalidade
statistic, p_valor = shapiro(dados_pacientes['Pressao_Arterial'])

# Verificar se rejeitamos ou não a hipótese nula
if p_valor < alpha:
    print("Rejeitamos a hipótese nula. A distribuição da pressão arterial na população não segue uma distribuição normal.")
else:
    print("Não rejeitamos a hipótese nula. A distribuição da pressão arterial na população segue uma distribuição normal.")


A versão B tem a maior taxa de conversão.
Tamanho da amostra necessário: 318
Não rejeitamos a hipótese nula. Não há evidências estatísticas suficientes para concluir que a idade média das pessoas com problemas cardíacos é maior que 50 anos.
Não rejeitamos a hipótese nula. Não há evidências de que a pressão arterial média para pacientes com condições de saúde adicionais é diferente da pressão arterial média para pacientes saudáveis.
Não rejeitamos a hipótese nula. Não há evidências de que há uma diferença significativa na pressão arterial média entre diferentes grupos étnicos nesta população.
Não rejeitamos a hipótese nula. Não há evidências de que há uma relação significativa entre o sexo e condições de saúde adicionais.
Não rejeitamos a hipótese nula. Não há evidências de que há uma associação entre a idade dos pacientes e sua pressão arterial.
Intervalo de confiança para a média da pressão arterial entre pacientes com condições de saúde adicionais: (137.95033343954205, 168.2290808434